In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq pydantic-ai==0.0.12 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.


In [35]:
!gdown 1zXnEv6yiQb39Mb9UQstHGx0vbcWAQHcM

Downloading...
From: https://drive.google.com/uc?id=1zXnEv6yiQb39Mb9UQstHGx0vbcWAQHcM
To: /content/app-reviews.db
100% 73.7k/73.7k [00:00<00:00, 94.8MB/s]


In [2]:
import os
import sqlite3
import textwrap
from dataclasses import dataclass
from pathlib import Path
from typing import List

import nest_asyncio
from google.colab import userdata
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

DB_PATH = Path("./app-reviews.db")
MODEL_NAME = "openai:gpt-4o-mini"

nest_asyncio.apply()

## Get Reviews

In [3]:
@dataclass
class Review:
    package_name: str
    text: str
    rating: int

In [4]:
def fetch_reviews(
    min_rating: int,
    max_rating: int,
    max_reviews: int = 30,
    min_words_in_review: int = 8,
) -> List[Review]:
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            SELECT package_name, review, rating
            FROM reviews
            WHERE rating >= ?
            AND rating <= ?
            AND review IS NOT NULL
            ORDER BY RANDOM()
""",
            (min_rating, max_rating),
        )
        reviews = []

        for package_name, review, rating in cursor.fetchall():
            word_count = len(review.split())
            if word_count < min_words_in_review:
                continue
            reviews.append(
                Review(package_name=package_name, text=review, rating=rating)
            )
        return reviews[:max_reviews]

In [5]:
reviews = fetch_reviews(2, 4, max_reviews=10)

In [6]:
reviews[0]

Review(package_name='org.isoron.uhabits', text="It's heartbreaking. Please respond. Please fix the Calendar not highlighting for mrasureable habits. The App is failing with most recent update 8/23/23. Measurable habits Issue with Calendar section blocks not highlighting days with mrasureable activity.", rating=2)

## Dependencies

In [ ]:
@dataclass
class Dependencies:
    app_description: str

## Agents

In [7]:
PRODUCT_IMPROVEMENT_PROMPT = """
You help app developers and product managers identify areas for improvement based
on user feedback.
""".strip()


class ProductImprovementResult(BaseModel):
    issues: List[str] = Field(
        description="Prioritized list of common bugs or issues (e.g., 'app crashes on startup')"
    )
    feature_requests: List[str] = Field(
        description="Suggest top requested features (e.g., 'add dark mode')"
    )


improvement_agent = Agent(
    MODEL_NAME,
    deps_type=Dependencies,
    result_type=ProductImprovementResult,
    system_prompt=PRODUCT_IMPROVEMENT_PROMPT,
)

In [8]:
MARKETING_PROMPT = """
You're marketer that focuses on engaging mobile app users.
You have a nag for writing clear and focused marketing copy.
""".strip()


class MarketingResult(BaseModel):
    praised_features: List[str] = Field(
        description="Prioritized list of features that users praise the most. . One item per line."
    )
    important_phrases: List[str] = Field(
        description="List of phrases that the marketing team should include in the copy. One item per line."
    )


marketing_agent = Agent(
    MODEL_NAME,
    deps_type=Dependencies,
    result_type=MarketingResult,
    system_prompt=MARKETING_PROMPT,
)

In [9]:
class PlanningResult(BaseModel):
    app_names: List[str] = Field(description="List of suggested app names")
    marketing_copy: str = Field(
        description="Marketing copy of the app to build. 2-3 paragraphs at most"
    )
    mvp_features: List[str] = Field(
        description="List of features to include in the MVP of the app"
    )
    possible_issues: List[str] = Field(
        description="Prioritized list of possible bugs that might come up during development"
    )


planner_agent = Agent(MODEL_NAME, deps_type=Dependencies, result_type=PlanningResult)

In [11]:
@planner_agent.system_prompt
async def get_system_prompt(ctx: RunContext[Dependencies]) -> str:
    return f"""
You're planning an MVP for new {ctx.deps.app_description} app. You'll propose
the best possible way to approach the process of building it.
""".strip()

## Tools

In [12]:
@improvement_agent.tool
def get_improvement_reviews(
    ctx: RunContext[Dependencies], min_rating: int, max_rating: int
) -> List[str]:
    """Get reviews for within a specified rating

    Args:
        min_rating: the low range of the ratings (inclusive). Between 1 and 5.
        max_rating: the high range of the ratings (inclusive). Between 1 and 5.
    """
    return fetch_reviews(min_rating, max_rating)

In [13]:
@marketing_agent.tool
def get_marketing_reviews(
    ctx: RunContext[Dependencies], min_rating: int, max_rating: int
) -> List[str]:
    """Get reviews for within a specified rating

    Args:
        min_rating: the low range of the ratings (inclusive). Between 1 and 5.
        max_rating: the high range of the ratings (inclusive). Between 1 and 5.
    """
    return fetch_reviews(min_rating, max_rating)

## Run Agents

In [14]:
deps = Dependencies(
    app_description="Todo/habit tracking app with focus on personal development"
)

In [15]:
%%time
improvement_result = improvement_agent.run_sync(
    "Analyze the reviews for possible improvements", deps=deps
)

CPU times: user 111 ms, sys: 12.5 ms, total: 123 ms
Wall time: 3.21 s


In [16]:
for item in improvement_result.new_messages():
    print(item)

UserPrompt(content='Analyze the reviews for possible improvements', timestamp=datetime.datetime(2024, 12, 12, 20, 40, 5, 521964, tzinfo=datetime.timezone.utc), role='user')
ModelStructuredResponse(calls=[ToolCall(tool_name='get_improvement_reviews', args=ArgsJson(args_json='{"min_rating": 1, "max_rating": 3}'), tool_id='call_YU3Zdp1ojtdklENRm2Q1lKOY'), ToolCall(tool_name='get_improvement_reviews', args=ArgsJson(args_json='{"min_rating": 4, "max_rating": 4}'), tool_id='call_GAMgfvZJiBINEMt3G0ODW89U')], timestamp=datetime.datetime(2024, 12, 12, 20, 40, 5, tzinfo=datetime.timezone.utc), role='model-structured-response')
ToolReturn(tool_name='get_improvement_reviews', content=[Review(package_name='org.isoron.uhabits', text="- The settings are limited, for the frequency of the habit, the options are only: everyday, every week, every month, NO SPECIFICATIONS. - No icons, it looks simple but I was hoping some added beauty to it. - Just read a review here and now I'm also scared of losing my d

In [17]:
for item in improvement_result.data.issues:
    print(f"- {item}")

- Data loss after one year of use
- Limited customization for habit reminders
- App crashes or freezes
- Lack of account transfer functionality between devices
- Poor widget functionality - frequently freezing
- Inadequate notification handling
- High premium pricing with limited features in the free version


In [18]:
for item in improvement_result.data.feature_requests:
    print(f"- {item}")

- Add ability to specify exact days for reminders
- Implement data retention beyond one year
- Introduce a simple design option without excessive animations
- Syncing capability across devices
- Landscaping mode support
- Option to assign a specific date to each task
- Add a daily percentage tracking feature


In [19]:
%%time
marketing_result = marketing_agent.run_sync(
    "Prepare marketing materials based on the reviews", deps=deps
)

CPU times: user 87.5 ms, sys: 3.22 ms, total: 90.8 ms
Wall time: 3.96 s


In [20]:
for item in marketing_result.new_messages():
    print(item)

UserPrompt(content='Prepare marketing materials based on the reviews', timestamp=datetime.datetime(2024, 12, 12, 20, 40, 17, 689082, tzinfo=datetime.timezone.utc), role='user')
ModelStructuredResponse(calls=[ToolCall(tool_name='get_marketing_reviews', args=ArgsJson(args_json='{"min_rating": 4, "max_rating": 5}'), tool_id='call_SdyOMr7F4DeRk4WCAjBj1bO4'), ToolCall(tool_name='get_marketing_reviews', args=ArgsJson(args_json='{"min_rating": 1, "max_rating": 3}'), tool_id='call_mXDntRxVCOFSEhTDW5BmIdWy')], timestamp=datetime.datetime(2024, 12, 12, 20, 40, 17, tzinfo=datetime.timezone.utc), role='model-structured-response')
ToolReturn(tool_name='get_marketing_reviews', content=[Review(package_name='com.finch.finch', text='Really good app. I love my micropet, and she provides a lot of self care facilities, which are helping me through dark days. Thank you so much to the developers for this light in the darkness 🙏😊', rating=5), Review(package_name='com.finch.finch', text="Been using this since

In [21]:
for item in marketing_result.data.praised_features:
    print(f"- {item}")

- Cute design and theme
- Self-care functionalities
- Effective for keeping on track
- Simple and straightforward task management
- Rewards and engaging elements
- Customization options
- Daily tracking
- Motivational elements


In [22]:
for item in marketing_result.data.important_phrases:
    print(f"- {item}")

- Daily self-care
- Stay on track every day
- Cute and fun way to manage tasks
- Turn mundane tasks into engaging activities
- Bring joy to personal development
- Motivate yourself with a micropet
- Effortlessly track habits and progress
- Simplified design for focus and clarity


In [23]:
%%time

prompt = f"""Propose an MVP based on these reports:

# Improvements:

common issues:
{improvement_result.data.issues}

feature requests:
{improvement_result.data.feature_requests}

# Marketing:

praised features:
{marketing_result.data.praised_features}

important phrases:
{marketing_result.data.important_phrases}
"""

planner_result = planner_agent.run_sync(prompt, deps=deps)

CPU times: user 36.1 ms, sys: 5.72 ms, total: 41.8 ms
Wall time: 2.9 s


In [24]:
for item in planner_result.data.app_names:
    print(item)

HabitPal
DailyJoy
TrackerBee
MotiVibe
SelfCareSync


In [34]:
for item in planner_result.data.marketing_copy.split("\n"):
    print(f"{textwrap.fill(item, 100)}")

Introducing DailyJoy, the cute and fun way to manage tasks and stay on track every day. With our
unique approach to personal development, you can effortlessly track your habits and progress while
enjoying a simplified design that brings joy to your self-care routine. Turn mundane tasks into
engaging activities and boost your motivation with our delightful micropet feature that keeps you
accountable and excited to achieve your goals.

DailyJoy is designed with you in mind, ensuring that your personal development journey is enjoyable
and straightforward. It offers the ability to customize reminders, track daily progress, and sync
seamlessly across devices, all wrapped in a charming and user-friendly interface. Let's make self-
care a joyous part of your daily life together!


In [26]:
for item in planner_result.data.mvp_features:
    print(f"- {item}")

- Ability to specify exact days for reminders
- Implement data retention beyond one year
- Simple design option without excessive animations
- Syncing capability across devices
- Landscaping mode support
- Assign a specific date to each task
- Daily percentage tracking feature


In [27]:
for item in planner_result.data.possible_issues:
    print(f"- {item}")

- Data loss after one year of use
- App crashes or freezes
- Lack of account transfer functionality between devices
- Poor widget functionality - frequently freezing
- Inadequate notification handling
- High premium pricing with limited features in the free version


In [28]:
planner_result.all_messages()

[SystemPrompt(content="You're planning an MVP for new Todo/habit tracking app with focus on personal development app. You'll propose\nthe best possible way to approach the process of building it.", role='system'),
 UserPrompt(content="Propose an MVP based on these reports:\n\n# Improvements:\n\ncommon issues: \n['Data loss after one year of use', 'Limited customization for habit reminders', 'App crashes or freezes', 'Lack of account transfer functionality between devices', 'Poor widget functionality - frequently freezing', 'Inadequate notification handling', 'High premium pricing with limited features in the free version']\n\nfeature requests:\n['Add ability to specify exact days for reminders', 'Implement data retention beyond one year', 'Introduce a simple design option without excessive animations', 'Syncing capability across devices', 'Landscaping mode support', 'Option to assign a specific date to each task', 'Add a daily percentage tracking feature']\n\n# Marketing:\n\npraised fe

## Chat History

In [29]:
%%time
planner_new_result = planner_agent.run_sync(
    "Focus the mvp features on the app functionality - habit tracking and tasks. Give a detailed explanation of the feature",
    message_history=planner_result.new_messages(),
    deps=deps,
)

CPU times: user 68.5 ms, sys: 2.36 ms, total: 70.8 ms
Wall time: 4.44 s


In [32]:
for item in planner_new_result.data.mvp_features:
    print(f"- {textwrap.fill(item, 100)}")
    print()

- User-friendly habit tracking interface: A straightforward dashboard where users can add, edit, or
delete habits and tasks, providing a visual overview of their progress and completion rates.

- Customizable reminders: Users can specify exact days and times for reminders, with options for
recurring alerts, ensuring they stay on track with their habits.

- Progress tracking through daily percentage: A feature that allows users to see how much of their
task they have completed each day, giving them insights into their behavior patterns and motivating
them to improve.

- Task assignment with specific due dates: Users can assign due dates to tasks to prioritize their
personal development goals, making it easier to plan and stay organized.

- Device synchronization: The app will support syncing across multiple devices, allowing users to
access their habits and tasks on their phones, tablets, or computers anytime, anywhere.

- Landscape mode support: A visually appealing layout that accommo